In [1]:
import os
import time

# pip/conda installed
import dask.array as da
import fsspec
import pandas as pd
import xarray as xr
from dask.distributed import as_completed
from dask.distributed import Client
from dask_gateway import GatewayCluster

from utils import get_logger
from utils.dask import create_cluster
from utils.dask import upload_source
from utils.hls.catalog import HLSBand
from utils.hls.catalog import HLSCatalog
from utils.hls.catalog import scene_to_urls
from utils.hls.compute import process_catalog
from utils.hls.compute import calculate_job_median

In [2]:
logger = get_logger('hls-wa-test')

In [3]:
# fill with your account key
os.environ['AZURE_ACCOUNT_KEY'] = ""

In [4]:
# read the entire data once (each tile is 3660x3660)...
chunks = {'band': 1, 'x': 3660, 'y': 3660}

In [5]:
catalog_url = fsspec.get_mapper(
    f"az://fia/catalogs/hls_wa_2015-2019.zarr",
    account_name="usfs",
    account_key=os.environ['AZURE_ACCOUNT_KEY']
)
catalog = HLSCatalog.from_zarr(catalog_url)

In [6]:
cluster = create_cluster(
    workers=6,
    worker_threads=1,
    worker_memory=6,
    scheduler_threads=4,
    scheduler_memory=8
)
client = cluster.get_client()
cluster

Cluster dashboard visible at: /services/dask-gateway/clusters/default.cbff43cbf5df421a9074d2144501651c/status


In [7]:
# All workers must be started when source code is uploaded to them.
upload_source('./', client)

In [8]:
account_name="usfs"
storage_container="fia/hls-testing/wa"
account_key=os.environ["AZURE_ACCOUNT_KEY"]
catalog_groupby = "tile"
job_groupby = "time.month"

In [9]:
# Filter to scenes from 2015 and later, then group by year
yr_catalogs = catalog.xr_ds.where(catalog.xr_ds['year'] >= 2017, drop=True).groupby('year')

In [11]:
for yr, ca in yr_catalogs:
        logger.info(f"Starting process for {yr}")
        ca.info()
        storage_prefix = f"{storage_container}/{yr}"
        process_catalog(
            catalog=ca,
            catalog_groupby=catalog_groupby,
            job_fn=calculate_job_median,
            job_groupby=job_groupby,
            chunks=chunks,
            account_name=account_name,
            storage_container=storage_prefix,
            account_key=account_key,
            client=client,
            concurrency=1,
            logger=logger
        )

2021-01-07 23:42:50,942 [INFO] hls-wa-test - Starting process for 2017.0
xarray.Dataset {
dimensions:
	index = 5041 ;

variables:
	datetime64[ns] dt(index) ;
	object scene(index) ;
	object sensor(index) ;
	object tile(index) ;
	float64 year(index) ;
	int64 index(index) ;

// global attributes:
	:bands = [<HLSBand.COASTAL_AEROSOL: 1>, <HLSBand.BLUE: 2>, <HLSBand.GREEN: 3>, <HLSBand.RED: 4>, <HLSBand.NIR_NARROW: 5>, <HLSBand.SWIR1: 6>, <HLSBand.SWIR2: 7>, <HLSBand.CIRRUS: 8>, <HLSBand.QA: 11>] ;
}2021-01-07 23:42:52,165 [INFO] hls-wa-test - Submitting job 10TCT
2021-01-07 23:43:05,903 [ERROR] hls-wa-test - Exception from dask cluster
Traceback (most recent call last):
  File "/home/jovyan/data-ingestion/utils/hls/compute.py", line 243, in process_catalog
    result = future.result()
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/distributed/client.py", line 224, in result
    raise exc.with_traceback(tb)
  File "/home/jovyan/dask-worker-space/dask-worker-space/worker-kp85j1

KeyboardInterrupt: 

In [12]:
cluster.shutdown()